In [ ]:
# import neccessary packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import glob
import os
import re
import random
import itertools
import collections
from collections import Counter
import regex as re
import nltk
from nltk.corpus import stopwords
from scipy.ndimage.filters import gaussian_filter1d
import seaborn as sns
import io
from google.colab import files
import xlrd
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize # nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import sys
print(sys.getrecursionlimit())

1000


In [ ]:
# had recursion max length error. now expanding the system limit 
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

In [ ]:
# 2. import data

df = pd.read_csv('YOUR FILE PATH')

df

,post_number,user_id,is_student,is_post,text_post,reply_number,Knowledge and content,competency and skills,application in the real world,jup
0,3023299.0,4547548.0,True,True,I need help with y=150+155x. Where y represent...,NaN,0.0,0.0,0.0,3023299.0
1,3023299.0,3615622.0,True,False,What is your question?,3023300.0,NaN,NaN,NaN,3023299.0
2,3023299.0,1957790.0,True,False,what is the question?,3023301.0,NaN,NaN,NaN,3023299.0
3,3023299.0,9854014.0,False,False,Amisadai please post the entire question so we...,3023302.0,NaN,NaN,NaN,3023299.0
4,3023299.0,4547548.0,True,False,Pete's Pest Control charges a one-time evaluat...,3023307.0,NaN,NaN,NaN,3023299.0
...,...,...,...,...,...,...,...,...,...,...
65330,952831.0,1516046.0,True,False,I'll be back on in 15 min I hope to see all of...,952944.0,NaN,NaN,NaN,952831.0
65331,952831.0,667659.0,True,False,That is right,952945.0,NaN,NaN,NaN,952831.0
65332,952831.0,3247136.0,True,False,date already lol,952947.0,NaN,NaN,NaN,952831.0
65333,952831.0,3719772.0,True,False,aww babe wait,952948.0,NaN,NaN,NaN,952831.0


In [ ]:
# within each group, merge multiple rows of the same speaker's uttences to one
# so the ABAAAB sequence will be treated as ABAB
for index, row in df.iterrows():
  # print(index)
  if index is not 0: 
    jup_prev = df.iloc[index-1]['jup']
  if((index == 0 or row['jup'] != jup_prev) and row['is_post'] == True): # If the current row is a new post. save the previous list, then initiate a new list.  
    # print("found a new post: ", row['jup'])
    student_id = row['user_id']
  # add a column about student/facilitator 
  if row['user_id'] == student_id:
    df.at[index,'user_role'] = 'student' 
  else:
    df.at[index,'user_role'] = 'facilitator' 
df['key'] = ((df['jup']!= df['jup'].shift(1))|
                (df['user_role']!= df['user_role'].shift(1))).astype(int).cumsum()



In [ ]:

grouped_df = df.groupby("key")
first_values = grouped_df.first().reset_index()

df_join_text = df.groupby(['key','user_role','jup'])['text_post'].apply(' '.join).to_frame().reset_index()
# print(df_join_text)
df_join_text
# df_join_text.to_excel('sample_merged.xlsx')

,key,user_role,jup,text_post
0,1,student,3023299.0,I need help with y=150+155x. Where y represent...
1,2,facilitator,3023299.0,What is your question? what is the question? A...
2,3,student,3023299.0,Pete's Pest Control charges a one-time evaluat...
3,4,facilitator,3023299.0,so first what would y int represent?
4,5,student,3023299.0,the total cost
...,...,...,...,...
33065,33066,facilitator,952831.0,"Now divide by 2 easy olivia, lets see if she d..."
33066,33067,student,952831.0,z&lt;13?
33067,33068,facilitator,952831.0,now divide both sides by 2 Great job!! You got...
33068,33069,student,952831.0,date already lol


## Adopted Text preprocessing

In [ ]:
# define the preprocessing functions: decontration, remove punctuations, remove stopwords
# also define a function to treat the math formula as a whole part. 
# In LAK, we recognize all math formula using regex and convert it to a single token "FFFF"

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

#print(decontracted("I won\'t tell you"))

def treat_formula_1(utterance):
      # replace operations with words
    utterance = re.sub(r'[+]', ' plus ', utterance) # '+'
    utterance = re.sub(r'[-]', ' minus ', utterance) # '-'
    utterance = re.sub(r'[*]', ' times ', utterance) # '*'
    if("axis" in utterance):
        utterance = re.sub(r'[x]', 'x', utterance) # 'x'
    else:
        utterance = re.sub(r'[x]', ' x', utterance) # 'x' # here is the issue 
    utterance = re.sub(r'[/]', ' divided by ', utterance) # '/'
    utterance = re.sub(r'[=]', ' equals to ', utterance) # '='
    return utterance


def treat_formula_2(utterance): # treat any formula as a universal word 
      # this regex captures the math formula (e.g., y=150+50x, 50+50x, m+n)
    utterance = re.sub(r'([a-zA-Z0-9])*\s?[-+/*=]\s?([\d+(\.\d+)?a-zA-Z])+\s?([-+/*=]\s?[\d+(\.\d+)?a-zA-Z])*', ' FFFF ', utterance) 

    return utterance


def text_preprocessing(utterance):
    # print("raw text is:", utterance)
    utterance = treat_formula_2(utterance)
    # replace all digital numbers with consistent string of 'number'
    # utterance = convert_num_to_words(utterance)
    # convert text to lowercase
    utterance = utterance.lower()
    # solve contractions
    utterance = decontracted(utterance)
    # remove '()' several times. '((())) happens in text corpus'
    utterance = re.sub('[()]', '', utterance)
    # remove '[]' and contents inside it
    utterance = re.sub("([\[]).*?([\]])", "", utterance)
    # remove specific symbols, keep ? and . because it conveys the question and answer info
    utterance = re.sub(r'[--]', '', utterance) # '--'
    utterance = re.sub(r'[\\]', '', utterance)  # '\'
    utterance = re.sub(r'[\']', '', utterance)  # '''
    utterance = re.sub(r'[!]', '', utterance)  # '!'
    utterance = re.sub(r'["]', '', utterance)  # '"'
    utterance = re.sub(r'[{]', '', utterance)  # '{,}'
    utterance = re.sub(r'[}]', '', utterance)  # '}'
    utterance = re.sub(r'[:]', '', utterance)  # ':'
    utterance = re.sub(r'[)]', '', utterance)  # ')'
    utterance = re.sub(r'[(]', '', utterance)  # '('
    utterance = re.sub(r'[.]', '', utterance)  # '.'
    utterance = re.sub(r'[&]', '', utterance)  # '&'
    utterance = re.sub(r'[$]', ' dollar ', utterance)  # '$'
    utterance = re.sub(r'[?]', '', utterance)  # '?'
    utterance = re.sub(r'[,]', '', utterance)  # ','

    utterance = re.sub(r'[äô]', '', utterance)  # ','
    utterance = re.sub(r'[‚Äô]', ' a', utterance)  # ','
    utterance = re.sub(r'[Ä¢]', ' ', utterance)  # what is this???
    # utterance = re.sub(r'[&quot;]', '', utterance)  # remove it?
        # utterance = re.sub()
    utterance = re.sub(r'[,]', '', utterance)  # ','
    # utterance = remove_stopwords(utterance)
    # remove '...'
    if '...' in utterance:
        utterance = utterance.replace('...', " ")
    # remove white space
    utterance = " ".join(utterance.split())
    # print("cleaned utterance is:", utterance)
    return utterance

# test the stopwords, stem methods
test_utterance = "So I would do the same for B value values valuable"


# remove stopwords, stemming using PorterStemmer  
def remove_stopwords_stemming(text): 
  stopword = stopwords.words('english')
  stemmer= nltk.PorterStemmer()
  text = [stemmer.stem(word) for word in text.split() if word not in stopword]
 # print(utterance)
  return text

print("test: ")
remove_stopwords_stemming(test_utterance)

test: 


['so', 'i', 'would', 'b', 'valu', 'valu', 'valuabl']

In [ ]:
## create the number list
def create_unique_word_list(word_list):
  unique_num = 1
  unique_word_list = []
  unique_num_list = []
  for word in word_list:
      if (word in unique_word_list):
          # print("repetive word detected!")
          unique_num += 0
      else:
          # print("unique word detected!")
          unique_word_list.append(word)
          unique_num_list.append(unique_num)
          unique_num += 1
  return unique_word_list


# Start from here

## Run text processing for each row in the df, write the clean text in a new column
### this will take a bit long


In [ ]:
# read file "post_full_merged.csv"
df_join_text = pd.read_csv('post_full_merged.csv')


## Convert the tokenized text `processed_text` to numerical list `num_list`

In [ ]:
# run text processing for each row in the df, write the clean text in a new column
df_join_text['processed_text'] = df_join_text['text_post']
df_join_text['len_text'] = 0
df_join_text['turn_id'] = df_join_text['key']
df_join_text['num_list'] = df_join_text['text_post']
corpus_word_dict = {} # key: value -> jup: conv_word_list
last_conv_len = 0
conv_count = 0
for conv_id in df_join_text['jup'].unique(): 
  # print(conv_id)
  conv_count += 1
  conv_word_list = []
  for i, utterance in enumerate(df_join_text[df_join_text['jup'] == conv_id]['processed_text']):
    # i is not good for tracking the index for the full dataset (it starts from 0 every loop). it needs to be added the prev length of the conv  
    i = i + last_conv_len 
    # print(last_conv_len, i, utterance)
    df_join_text['processed_text'][i] = remove_stopwords_stemming(text_preprocessing(utterance))
    df_join_text['len_text'][i] = len(remove_stopwords_stemming(text_preprocessing(utterance)))
    # add the words to the corpus to assign unique numbers later on
    conv_word_list += remove_stopwords_stemming(text_preprocessing(utterance))
  # print('done processing conv ', conv_id)
  # print('conv: ', conv_id, conv_word_list)
  corpus_word_dict[conv_id] = conv_word_list # save it to a dict. but won't use 
  unique_word_list = create_unique_word_list(conv_word_list)
  for i, utterance in enumerate(df_join_text[df_join_text['jup'] == conv_id]['processed_text']):
    # print(i, utterance)
    i = i + last_conv_len 
    df_join_text['num_list'][i] = []
    # df_join_text['turn_id'][i] = math.ceil(df_join_text['key'][i]/2)  # do not trust the turn_id here 
    for word in utterance: 
      # print(word)
      # print(unique_word_list.index(word)+1)
      unique_num = unique_word_list.index(word) + 1
      df_join_text['num_list'][i].append(unique_num)
  # keep track of the row index 
  last_conv_len = i+1
  # add the number list as a separate column 
  if conv_count % 500 == 0:
    print('Processed post counts: ', conv_count)




df_join_text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processed post counts:  500


,Unnamed: 0,key,user_role,jup,text_post,processed_text,len_text,turn_id,num_list
0,0,1,student,3023299.0,I need help with y=150+155x. Where y represent...,"[need, help, ffff, repres, total, cost]",6,1,"[1, 2, 3, 4, 5, 6]"
1,1,2,facilitator,3023299.0,What is your question? what is the question? A...,"[question, question, amisadai, pleas, post, en...",8,2,"[7, 7, 8, 9, 10, 11, 7, 2]"
2,2,3,student,3023299.0,Pete's Pest Control charges a one-time evaluat...,"[pete, pest, control, charg, ffff, evalu, fee,...",17,3,"[12, 13, 14, 15, 3, 16, 17, 5, 6, 18, 19, 3, 2..."
3,3,4,facilitator,3023299.0,so first what would y int represent?,"[first, would, int, repres]",4,4,"[24, 25, 26, 4]"
4,4,5,student,3023299.0,the total cost,"[total, cost]",2,5,"[5, 6]"
...,...,...,...,...,...,...,...,...,...
33065,33065,33066,facilitator,952831.0,"Now divide by 2 easy olivia, lets see if she d...","[divid, 2, easi, olivia, let, see, step, right]",8,33066,"[61, 62, 71, 72, 73, 33, 74, 42]"
33066,33066,33067,student,952831.0,z&lt;13?,[zlt;13],1,33067,[75]
33067,33067,33068,facilitator,952831.0,now divide both sides by 2 Great job!! You got...,"[divid, side, 2, great, job, got, good, perfec...",20,33068,"[61, 7, 62, 76, 77, 23, 78, 79, 15, 80, 19, 81..."
33068,33068,33069,student,952831.0,date already lol,"[date, alreadi, lol]",3,33069,"[86, 87, 18]"


In [ ]:
df_join_text.to_csv('temp_join_text.csv')

## Equalize the utternace length for each turn 

In [ ]:
# build correct turn id

df_join_text['turn_id'] = df_join_text.groupby(['jup','user_role']).cumcount()+1
df_join_text['conv_utternace_id'] = df_join_text.groupby(['jup']).cumcount()+1

df_join_text[:10]


,Unnamed: 0,key,user_role,jup,text_post,processed_text,len_text,turn_id,num_list,conv_utternace_id
0,0,1,student,3023299.0,I need help with y=150+155x. Where y represent...,"[need, help, ffff, repres, total, cost]",6,1,"[1, 2, 3, 4, 5, 6]",1
1,1,2,facilitator,3023299.0,What is your question? what is the question? A...,"[question, question, amisadai, pleas, post, en...",8,1,"[7, 7, 8, 9, 10, 11, 7, 2]",2
2,2,3,student,3023299.0,Pete's Pest Control charges a one-time evaluat...,"[pete, pest, control, charg, ffff, evalu, fee,...",17,2,"[12, 13, 14, 15, 3, 16, 17, 5, 6, 18, 19, 3, 2...",3
3,3,4,facilitator,3023299.0,so first what would y int represent?,"[first, would, int, repres]",4,2,"[24, 25, 26, 4]",4
4,4,5,student,3023299.0,the total cost,"[total, cost]",2,3,"[5, 6]",5
5,5,6,facilitator,3023299.0,"not quite, that is what y represents What is ...","[quit, repres, valu, ffff, think, quot;on, tim...",28,3,"[27, 4, 28, 3, 29, 30, 31, 16, 32, 33, 34, 35,...",6
6,6,7,student,3023299.0,they have to pay more because of the fee yes t...,"[pay, fee, ye, multipl, choic, option]",6,4,"[44, 17, 46, 34, 35, 36]",7
7,7,8,facilitator,3023299.0,Can you post them? not neccisarily before we d...,"[post, neccisarili, anyth, alreadi, pay, ___, ...",16,4,"[10, 47, 40, 48, 44, 49, 26, 50, 51, 52, 53, 5...",8
8,8,9,student,3023299.0,i already did,[alreadi],1,5,[48],9
9,9,10,facilitator,3023299.0,the one time evaluation fee would be the first...,"[one, time, evalu, fee, would, first, fee, oka...",9,5,"[42, 31, 16, 17, 25, 24, 17, 58, 59]",10


In [ ]:
## equalize the utternace length for each turn 
df_join_text['max_len'] = 0
last_index_conv = max(df_join_text['key'])-1 
for i, turn_id in enumerate(df_join_text['turn_id']):
  # print('i: ', i)
  # keep track of conversation id, speaker, and turn id 
  conv_id = df_join_text['jup'][i]
  speaker = df_join_text['user_role'][i]
  # print(last_index_conv)
  # print('current utterance: ', df_join_text['len_text'][i])
  if i==last_index_conv: 
    print('break here. end of one conversation', i)
    break 
  # first utternace of a turn, not the last utternace of a conversation 
  elif (conv_id == df_join_text['jup'][i+1] and turn_id == df_join_text['turn_id'][i+1]): 
    # print('next utterance: ', df_join_text['len_text'][i+1])
    max_len = max(df_join_text['len_text'][i],df_join_text['len_text'][i+1])
    # print('max: ', max_len)
    df_join_text['max_len'][i] = max_len
    df_join_text['max_len'][i+1] = max_len


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


break here. end of one conversation 33069


In [ ]:
# add padding to the short utterances to equalize the num_list
df_join_text['num_list_equal'] = df_join_text['num_list']
for i, len_text in enumerate(df_join_text['len_text']):
  # if a turn only have one utterance (usually the fianl utterance of a post, this utterance does not have anything to repeate for)
  if df_join_text['max_len'][i] == 0: 
    df_join_text['num_list_equal'][i] = []
  elif len_text < df_join_text['max_len'][i]:
    #  dummy = [-1] * 5 will be [-1, -1, -1, -1, -1]
    length = df_join_text['max_len'][i]-len_text
    # print(length)
    # set dummy to be the negative value of the 'key' so it won't accidentally overlap with anything else
    dummy = [int(df_join_text['key'][i])*-1]* length
    # print('dummy: ', dummy)
    # print(type(dummy))
    df_join_text['num_list_equal'][i] = df_join_text['num_list_equal'][i] + dummy
  # keep track of conversation id, speaker, and turn id 
df_join_text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,key,user_role,jup,text_post,processed_text,len_text,turn_id,num_list,conv_utternace_id,max_len,num_list_equal
0,0,1,student,3023299.0,I need help with y=150+155x. Where y represent...,"[need, help, ffff, repres, total, cost]",6,1,"[1, 2, 3, 4, 5, 6]",1,8,"[1, 2, 3, 4, 5, 6, -1, -1]"
1,1,2,facilitator,3023299.0,What is your question? what is the question? A...,"[question, question, amisadai, pleas, post, en...",8,1,"[7, 7, 8, 9, 10, 11, 7, 2]",2,8,"[7, 7, 8, 9, 10, 11, 7, 2]"
2,2,3,student,3023299.0,Pete's Pest Control charges a one-time evaluat...,"[pete, pest, control, charg, ffff, evalu, fee,...",17,2,"[12, 13, 14, 15, 3, 16, 17, 5, 6, 18, 19, 3, 2...",3,17,"[12, 13, 14, 15, 3, 16, 17, 5, 6, 18, 19, 3, 2..."
3,3,4,facilitator,3023299.0,so first what would y int represent?,"[first, would, int, repres]",4,2,"[24, 25, 26, 4]",4,17,"[24, 25, 26, 4, -4, -4, -4, -4, -4, -4, -4, -4..."
4,4,5,student,3023299.0,the total cost,"[total, cost]",2,3,"[5, 6]",5,28,"[5, 6, -5, -5, -5, -5, -5, -5, -5, -5, -5, -5,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
33065,33065,33066,facilitator,952831.0,"Now divide by 2 easy olivia, lets see if she d...","[divid, 2, easi, olivia, let, see, step, right]",8,7,"[61, 62, 71, 72, 73, 33, 74, 42]",14,8,"[61, 62, 71, 72, 73, 33, 74, 42]"
33066,33066,33067,student,952831.0,z&lt;13?,[zlt;13],1,8,[75],15,20,"[75, -33067, -33067, -33067, -33067, -33067, -..."
33067,33067,33068,facilitator,952831.0,now divide both sides by 2 Great job!! You got...,"[divid, side, 2, great, job, got, good, perfec...",20,8,"[61, 7, 62, 76, 77, 23, 78, 79, 15, 80, 19, 81...",16,20,"[61, 7, 62, 76, 77, 23, 78, 79, 15, 80, 19, 81..."
33068,33068,33069,student,952831.0,date already lol,"[date, alreadi, lol]",3,9,"[86, 87, 18]",17,7,"[86, 87, 18, -33069, -33069, -33069, -33069]"


In [ ]:
df_join_text.to_csv('full_post_processed_equalized.csv')

In [ ]:
df_join_text.to_pickle('full_post_processed_equalized.pkl')

## Aggregate the utterance-level num_list to conversational-level

In [ ]:
df_sum = df_join_text.groupby(['jup','user_role']).agg({'num_list_equal': sum})
df_sum = df_sum[df_sum['num_list_equal'].map(lambda d: len(d)) > 0]

In [ ]:
len(df_join_text['jup'].unique())

776

In [ ]:
df_sum.to_csv('sum_full.csv')

# NOW witch to R to compute the RQA indices
